In [1]:
import json
import os
import numpy as np
import tensorflow as tf
from pathlib2 import Path
import preprocess
from data_loader import create_dataset, create_inputs

2021-12-03 17:36:37.251265: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-03 17:36:37.251379: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def prepare_data(data_dir, language):
    if language == "Chinese":
        train_path = data_dir / "train_cn.json"
        test_path = data_dir / "test_cn.json"
        dev_path = data_dir / "dev_cn.json"

    elif language == "English":
        train_path = data_dir / "train_en.json"
        test_path = data_dir / "test_en.json"
        dev_path = data_dir / "dev_en.json"

    else:
        raise ValueError("Language must be English or Chinese")

    if not os.path.isfile(test_path):
        test_path = None

    return train_path, dev_path, test_path

data_dir = "./dataset"
language = "Chinese"
train_path, dev_path, test_path = prepare_data(data_dir=Path(data_dir), language=language)
train_path, dev_path, test_path

(PosixPath('dataset/train_cn.json'), PosixPath('dataset/dev_cn.json'), None)

In [3]:
data1  = create_inputs(json_path=train_path, plm="BERT", max_len=512, is_train=True, language="Chinese",
                      task="nugget")
train_dataset = create_dataset(data1, "nugget", 10000)
a = list(train_dataset.take(1))

2021-12-03 17:36:38.283602: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-03 17:36:38.283663: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-03 17:36:38.283677: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-EFG9I9T): /proc/driver/nvidia/version does not exist
2021-12-03 17:36:38.283875: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-03 17:36:48.324211: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:380] Filling up shuffle bu

In [4]:
input_ids = a[0]["input_ids"]
input_mask = a[0]["input_mask"]
input_type_ids = a[0]["input_type_ids"]
sentence_ids = a[0]["sentence_ids"]
sentence_masks = a[0]["sentence_masks"]
inputs = {
        "input_ids": input_ids, "input_mask": input_mask, "input_type_ids": input_type_ids,
        "sentence_ids": sentence_ids, "sentence_masks": sentence_masks
    }
from model import Bert
plm = Bert(language=language, embedding_size=0)

x = plm(inputs)

Some layers from the model checkpoint at bert-base-chinese were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-chinese.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [5]:
from model import TransformerEncoder
encoder = TransformerEncoder(d_model=768, num_heads=8, d_ff=200, maximum_position_encoding=5000, rate=0.1, num_layers=1)
test = encoder(x, training=True, mask=sentence_masks)

(16, 7)


In [6]:
test.shape

TensorShape([16, 7, 768])

In [7]:
from model import CustomDense
sentence_masks = a[0]["sentence_masks"]
dense = CustomDense(4, 3, 7)
clogits, hlogits = dense(test, mask=sentence_masks)

In [8]:
customer_labels = a[0]["customer_labels"]
helpdesk_labels = a[0]["helpdesk_labels"]
inputs2 = {"customer_labels": customer_labels, "helpdesk_labels": helpdesk_labels,
          "customer_logits": clogits, "helpdesk_logits": hlogits}

In [9]:
from model import CustomSoftmax
custom_softmax = CustomSoftmax(max_turn_number=7)
c_prob, h_prob = custom_softmax(inputs=inputs2, mask=sentence_masks)

(16, 4) (16, 3)
tf.Tensor(
[4.878669  2.8054245 4.329717  3.1554027 5.7458534 1.7262712 4.0557
 3.5113597 1.9150896 3.3787055 3.025288  1.6535207 4.341744  2.8330867
 1.3862944 5.3751006], shape=(16,), dtype=float32) tf.Tensor(
[5.974622  3.8588386 3.2730818 4.350981  6.7058606 1.304442  4.158972
 5.7255797 1.5271741 3.9570093 4.4696436 1.4189701 4.5971127 1.350988
 1.0688771 3.5857024], shape=(16,), dtype=float32)
xxx tf.Tensor(6.9653177, shape=(), dtype=float32)
tf.Tensor(
[[[0.21141364 0.3048147  0.21141364 0.272358  ]
  [0.1992519  0.32062694 0.14135268 0.3387685 ]
  [0.14752297 0.39770296 0.14752297 0.3072511 ]
  [0.09462867 0.6295936  0.09462867 0.18114896]]

 [[0.2691574  0.40651464 0.16216394 0.16216394]
  [0.25       0.25       0.25       0.25      ]
  [0.         0.         0.         0.        ]
  [0.         0.         0.         0.        ]]

 [[0.2276972  0.30551267 0.23909299 0.2276972 ]
  [0.15884523 0.1766951  0.50561446 0.15884523]
  [0.16188608 0.38421854 0.23544784 

In [ ]:
### log2(0)

In [ ]:
cprob = tf.nn.softmax(clogits)
clabels = a[0]["customer_labels"]

In [ ]:
cprob.shape, clabels.shape

In [ ]:
squared_difference = tf.math.squared_difference(
    cprob, clabels, name=None
)
squared_difference

In [ ]:

squared_difference

In [ ]:
mid = tf.reduce_sum(squared_difference, axis=-1)
mid

In [ ]:
aa = -tf.experimental.numpy.log2(tf.math.sqrt(mid / 2))
aa   

In [ ]:
xx = aa * tf.cast(cust_mask, tf.float32)
xx

In [ ]:
bb = tf.math.count_nonzero(xx, axis=-1)
bb

In [ ]:
tf.math.divide(tf.reduce_sum(xx, axis=-1), tf.cast(bb,dtype=tf.float32))

In [ ]:
tf.reduce_mean(tf.math.divide(tf.reduce_sum(xx, axis=-1), tf.cast(bb,dtype=tf.float32)))

In [ ]:
tf.reduce_mean(aa, axis=-1)

In [ ]:
tf.reduce_mean(xx, axis=-1)

In [ ]:
r = tf.reduce_sum(mid, axis=-1)
from math import log2
-log2(tf.reduce_mean(tf.math.sqrt(r/2)))

In [ ]:
cus_rnss = tf.math.sqrt(tf.reduce_sum(tf.reduce_sum(squared_difference, axis=-1), axis=-1) / 2)
-tf.experimental.numpy.log2(cus_rnss)

In [ ]:
def squared_error(pred, truth):
    return ((pred - truth) ** 2)

cprob_numpy = cprob.numpy()
ctruth_numpy = clabels.numpy()
squared_error(cprob_numpy, ctruth_numpy) * cust_mask[:, :, None].numpy()